In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

In [2]:
metadata = pd.read_csv('movies_metadata.csv', low_memory = False)
# metadata.head().T

In [3]:
metadata.drop(metadata.index[metadata['id'] == '1997-08-20'], inplace = True)

In [4]:
metadata.drop(metadata.index[metadata['id'] == '2012-09-29'], inplace = True)

In [5]:
metadata.drop(metadata.index[metadata['id'] == '2014-01-01'], inplace = True)

In [6]:
metadata['id'] = metadata['id'].astype(str).astype(int)

In [7]:
cred = pd.read_csv('credits.csv')
# cred.head().T

In [8]:
metadata= metadata.merge(cred,on='id')
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [9]:
metadata = metadata.drop(['belongs_to_collection', 'homepage', 'poster_path'], axis=1)

In [10]:
metadata['budget'] = metadata['budget'].astype(str).astype(float)

In [11]:
metadata['genres'] = metadata['genres'].apply(literal_eval)

In [12]:
gen = []

for i in metadata['genres']:
    l = []
    
    for j in i:
        l.append(j['name'])
#         print(j['name'])
    gen.append(l)

In [13]:
metadata['genres'] = pd.Series(gen)

In [14]:
metadata['cast'] = metadata['cast'].apply(literal_eval)

In [15]:
actors = []

for i in metadata['cast']:
    l = []
    for j in i:
        l.append(j['name'])
    
    actors.append(l)

In [16]:
metadata['actors'] = pd.Series(actors)

In [17]:
metadata['crew']= metadata['crew'].apply(literal_eval)

In [18]:
director = []
c = 0

for i in metadata['crew']:
    l = []
    for x in i:
        if x['job'] =='Director':
            l.append(x['name'])
#     print(l)
    director.append(l)


In [19]:
metadata['directors'] = pd.Series(director)

In [20]:
df = metadata.copy()

In [21]:
C = df['vote_average'].mean()
C

5.61823212913114

In [22]:
M = df['vote_count'].quantile(0.9)
M

160.0

In [23]:
simple = df.copy().loc[df['vote_count'] >= M]
simple.shape

(4563, 25)

In [24]:
def weightedRating(x, m =M, c=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [25]:
simple['rating_score'] = df.apply(weightedRating, axis = 1)

In [26]:
simple = simple.sort_values('rating_score', ascending = False)

In [43]:
simple[['title', 'vote_average', 'rating_score', 'actors']]

,title,vote_average,rating_score,actors
314,The Shawshank Redemption,8.5,8.445870,"[Tim Robbins, Morgan Freeman, Bob Gunton, Clan..."
837,The Godfather,8.5,8.425439,"[Marlon Brando, Al Pacino, James Caan, Richard..."
10345,Dilwale Dulhania Le Jayenge,9.1,8.421458,"[Shah Rukh Khan, Kajol, Amrish Puri, Anupam Kh..."
12525,The Dark Knight,8.3,8.265477,"[Christian Bale, Michael Caine, Heath Ledger, ..."
2854,Fight Club,8.3,8.256385,"[Edward Norton, Brad Pitt, Meat Loaf, Jared Le..."
...,...,...,...,...
9746,Son of the Mask,3.6,4.238176,"[Jamie Kennedy, Alan Cumming, Traylor Howard, ..."
12955,Disaster Movie,3.1,4.082725,"[Matt Lanter, Vanessa Lachey, Nicole Ari Parke..."
3482,Battlefield Earth,3.0,3.999802,"[John Travolta, Barry Pepper, Forest Whitaker,..."
11602,Epic Movie,3.2,3.983233,"[Kal Penn, Adam Campbell, Jennifer Coolidge, J..."
